## 📅 Dia 3 – Classificação de Texto com Embedding + Dense

### 🎯 Objetivo:
Aprender a tratar texto como entrada para redes neurais e aplicar em um caso real (sentimentos).

### 📚 Teoria:
- Pré-processamento de texto: tokenização, padding
- Embeddings: transformar palavras em vetores
- Arquitetura comum: Embedding → Flatten → Dense

### 🛠 Prática:
- Dataset: IMDb (crítico de cinema – positivo/negativo)
- Usar Tokenizer, pad_sequences
- Criar rede com:
    - Embedding(input_dim, output_dim)
    - Dense final com ativação sigmoid
    - Avaliar o modelo com precisão e matriz de confusão

---

# **Tokenização**
Transformar palavras em números, para que o modelo consiga compreender melhor.

### Exemplo:

In [113]:
frases = ["Eu gosto de gatos", "Eu não gosto de tigres"]

#Simulando um vocabulário
vocabulario = { "Eu" : 1,
                "gosto" : 2,
                "de" : 3, 
                "gatos" : 4,
                "não"   : 5,
                "tigres" : 6
                }

#Tokenizando cada frase em IDs
tokens = []
for frase in frases:
    token_frase = []
    for palavra in frase.split():
        token_frase.append(vocabulario[palavra])
    tokens.append(token_frase)

print("Tokens: ", tokens)

Tokens:  [[1, 2, 3, 4], [1, 5, 2, 3, 6]]


---
# **Padding**
Preenche espaços vazios com 0 para que as listas de números formados por cada frase fique do mesmo tamanho, pois os modelos esperam sempre entradas com o mesmo tamanho.

In [114]:
import torch
from torch.nn.utils.rnn import pad_sequence

In [115]:
# Convertendo listas de tokens em tensores individuais
tensor_tokens = []
for token in tokens:
    tensor_tokens.append(torch.tensor(token))

padded_sequences = pad_sequence(tensor_tokens, batch_first=True)

print(padded_sequences)

tensor([[1, 2, 3, 4, 0],
        [1, 5, 2, 3, 6]])


---

# **Embeddings**
Representações vetoriais de palavras ou tokens, com o objetivo de transformar essas strings ou tokens em vetores contínuos em um espaço de alta dimensão com propriedades que capturam o significado e a semelhança entre as palavras.

A camada ``nn.Embedding`` vai mapear esses índices(do vocabulario) para vetores.

### **Exemplo:** 

Passo a passo:
- Criar um vocabulário (um dicionário de palavras e seus índices).
- Criar a camada de Embedding com a classe nn.Embedding.
- Converter as palavras para índices.
- Passar os índices pela camada de embedding para obter os vetores.

In [116]:
import torch.nn as nn

#vocabulário já definido anteriormente

In [117]:
camada_embedding = nn.Embedding(num_embeddings=len(vocabulario)+1, embedding_dim=3) # (5 palavras no vocabulário, com vetores de tamanho 3)

In [118]:
for seq in padded_sequences:
    print(camada_embedding(seq))

tensor([[-0.4245,  0.3057, -0.7360],
        [-0.8371, -0.9224,  1.8113],
        [ 0.1606,  0.3672,  0.1754],
        [ 1.3852,  1.3835, -1.2024],
        [-0.2234,  1.7174,  0.3189]], grad_fn=<EmbeddingBackward0>)
tensor([[-0.4245,  0.3057, -0.7360],
        [ 0.7078, -1.0759,  0.5357],
        [-0.8371, -0.9224,  1.8113],
        [ 0.1606,  0.3672,  0.1754],
        [ 1.1754,  0.5612, -0.4527]], grad_fn=<EmbeddingBackward0>)


--- 

### **Plotando em 3D para verificar as semelhanças**

In [119]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [120]:
torch.manual_seed(42)

animais = ["gato", "cachorro", "tigre", "leão", "pratos"] #pratos para verificar o quão longe fica

#Tokenização
tokens_animais = {  "gato" : 1,
                    "cachorro" : 2,
                    "tigre" : 3,
                    "leão" : 4,
                    "pratos" : 5
                    }

sequencias = [["gato", "cachorro"], ["tigre", "leão"], ["tigre", "pratos"]] #como se fossem frases, para ter um contexto onde eles estariam juntos

tokens_map = []
for sequencia in sequencias:
    tokens_seq = []
    for palavra in sequencia:
        tokens_seq.append(tokens_animais[palavra])
    tokens_map.append(tokens_seq) #gato e cachorro = [1, 2] | tigre e leao = [3,4] | tigre e prato = [3,5]

#transformando as listas de tokens em tensores
tensor_tokens_animais = []
for token in tokens_map: #tokens_map pois é onde eles já estão separados por contexto
    tensor_tokens_animais.append(torch.tensor(token, dtype=torch.long))

#Padding
padded_sequences_animais = pad_sequence(tensor_tokens_animais, batch_first=True)

#Embbedding
embeddings  = nn.Embedding(num_embeddings=len(tokens_animais)+1, embedding_dim=3)

embeddings_list = []
for seq in padded_sequences_animais:
    embeddings_list.append(embeddings(seq))
    print(embeddings(seq))


tensor([[ 0.4396, -0.7581,  1.0783],
        [ 0.8008,  1.6806,  0.3559]], grad_fn=<EmbeddingBackward0>)
tensor([[-0.6866,  0.6105,  1.3347],
        [-0.2316,  0.0418, -0.2516]], grad_fn=<EmbeddingBackward0>)
tensor([[-0.6866,  0.6105,  1.3347],
        [ 0.8599, -0.3097, -0.3957]], grad_fn=<EmbeddingBackward0>)


In [121]:
embeddings_tensor = torch.cat(embeddings_list, dim=0)

### **Criando o Gráfico 3D**

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Acessando as colunas de embeddings para o gráfico 3D
ax.scatter(embeddings_tensor[:, 0].detach().numpy(), 
           embeddings_tensor[:, 1].detach().numpy(), 
           embeddings_tensor[:, 2].detach().numpy())

# Adicionando texto ao gráfico para cada ponto
for i in range(len(animais)):
    x = embeddings_tensor[i, 0].item()
    y = embeddings_tensor[i, 1].item()
    z = embeddings_tensor[i, 2].item()
    ax.text(x, y, z, animais[i])

plt.title("Visualização 3D dos Embeddings")
plt.show()

--- 

# **Arquiteturas**

---

# **Prática**

- Dataset: IMDb (crítico de cinema – positivo/negativo)
- Usar Tokenizer, pad_sequences
- Criar rede com:
    - Embedding(input_dim, output_dim)
    - Dense final com ativação sigmoid
    - Avaliar o modelo com precisão e matriz de confusão